# **Proyecto 5 - Reinforcement Learning - MÁSTER EN DATA SCIENCE & AI : NUCLIO**

# Actividad Reinforcemente Learning **Frozen lake problem**

Resolver el problema del Frozen lake de OpenAI Gym. Documentación: https://www.gymlibrary.dev/environments/toy_text/frozen_lake/

## Objetivos
- Conseguir movermos aleatoriamente hasta cumplir el objetivo
- Conseguir que el agente aprenda con Q-learning
- (Opcional) Probar con otros hiperparámetros
- (Opcional) Modificar la recompensa

## Consideraciones
- No hay penalizaciones
- Si el agente cae en un "hole", entonces done = True y se queda atascado sin poder salir (al igual que ocurre cuando llega al "goal")

## Normas a seguir

- Se debe entregar un **ÚNICO GOOGLE COLAB notebook** (archivo .ipynb) que incluya las instrucciones presentes y su **EJECUCIÓN!!!**.
- Poner el nombre del grupo en el nombre del archivo y el nombre de todos los integrantes del grupo al inicio del notebook.

## Criterio de evaluación

- Seguimiento de las normas establecidas en la actividad.
- Corrección en el uso de algoritmos, modelos y formas idiomáticas en Python.
- El código debe poder ejecutarse sin modificación alguna en Google Colaboratory.

## **Instalamos librerías**

In [5]:
pip install gym

In [6]:
import gym
import numpy as np
from time import sleep
from IPython.display import clear_output
import random as rd

## **Definición del entorno**

In [7]:
# Definimos el entorno
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)

In [8]:
# Fijamos una semilla
seed_value = 42
env.seed(seed_value)
np.random.seed(seed_value)

/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [9]:
env.reset() # En este caso, empieza desde la misma posición inicial
print(env.render())

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


None


In [10]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(4)
State Space Discrete(16)


Acciones posibles:
* 0: izquierda
* 1: abajo
* 2: derecha
* 3: arriba

In [11]:
# Identificador de estado
state = env.s
print("State:", state)

State: 0


## **¡Nos movemos aleatoriamente!**

In [12]:
steps = 0
env.reset()
env.render()

In [13]:
# Acciones: 0=izquierda, 1=abajo, 2=derecha, 3=arriba
action = 1
state, reward, done, info = env.step(action)

print("State:", state)
print(state, reward, done, info)

env.s = state
env.render()

steps += 1

print(f"Step: {steps}")

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


State: 4
4 0.0 False {'prob': 1.0}
Step: 1


## **Resolución del problema**

In [14]:
# Parámetros del algoritmo Q-Learning
alpha = 0.1         # Tasa de aprendizaje
gamma = 0.99        # Factor de descuento
epsilon = 1.0       # Valor inicial de epsilon para exploración
epsilon_min = 0.01  # Mínimo valor de epsilon
epsilon_decay = 0.999 # Tasa de decremento de epsilon
episodes = 20000     # Número de episodios para entrenamiento
max_steps = 100     # Máximo número de pasos por episodio

In [15]:
# Inicializamos la tabla Q con ceros
q_table = np.zeros((env.observation_space.n, env.action_space.n))

In [16]:
# Función para seleccionar una acción usando la política epsilon-greedy
def choose_action(state):
    if np.random.rand() < epsilon:
        return env.action_space.sample()  # Exploración: selecciona una acción aleatoria
    else:
        return np.argmax(q_table[state])  # Explotación: selecciona la mejor acción según la tabla Q

In [17]:
# Entrenamiento del agente
for episode in range(episodes):
    state = env.reset()  # Reinicia el entorno y obtiene el estado inicial
    done = False
    step = 0
    global epsilon

    for step in range(max_steps):
        action = choose_action(state)

        # Tomamos la acción en el entorno
        next_state, reward, done, _ = env.step(action)

        # Actualizamos la tabla Q usando la ecuación de Q-Learning
        q_value = q_table[state, action]
        max_q_value_next = np.max(q_table[next_state])  # Mejor estimación futura
        q_table[state, action] = q_value + alpha * (reward + gamma * max_q_value_next - q_value)

        # Actualizamos el estado
        state = next_state

        # Si llegamos al objetivo o caemos en un hueco, terminamos el episodio
        if done:
            break

    # Reducimos epsilon (para disminuir la exploración)
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# Mostrar la tabla Q entrenada
print("Tabla Q final:")
print(q_table)

Tabla Q final:
[[0.94148015 0.93206535 0.95099005 0.94148015]
 [0.94148015 0.         0.96059601 0.95099005]
 [0.95099005 0.970299   0.95099005 0.96059601]
 [0.96059601 0.         0.90255331 0.86023071]
 [0.89430452 0.73695422 0.         0.94148015]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.         0.96059601]
 [0.         0.         0.         0.        ]
 [0.23436875 0.         0.92890457 0.62118314]
 [0.68788948 0.67713159 0.9801     0.        ]
 [0.97029895 0.99       0.         0.970299  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.50480754 0.9899999  0.5589465 ]
 [0.98009861 0.99       1.         0.9801    ]
 [0.         0.         0.         0.        ]]


In [18]:
# Probar el agente
state = env.reset()
env.render()
done = False
steps = 0

print("\n--- Prueba del agente entrenado ---")
while not done:
    # Elegimos la acción óptima (sin exploración)
    action = np.argmax(q_table[state])
    next_state, reward, done, _ = env.step(action)

    # Mostramos el entorno
    env.render()
    print(f"Step: {steps}, State: {state}, Action: {action}, Reward: {reward}, Done: {done}")

    # Actualizamos el estado
    state = next_state
    steps += 1

    # Si el agente cae en un hueco o alcanza el objetivo, termina la prueba
    if done:
        break

# Imprimimos el resultado según el último estado
if reward == 1:
    print("Prueba conseguida")
else:
    print("Te caíste")


--- Prueba del agente entrenado ---
Step: 0, State: 0, Action: 2, Reward: 0.0, Done: False
Step: 1, State: 1, Action: 2, Reward: 0.0, Done: False
Step: 2, State: 2, Action: 1, Reward: 0.0, Done: False
Step: 3, State: 6, Action: 1, Reward: 0.0, Done: False
Step: 4, State: 10, Action: 1, Reward: 0.0, Done: False
Step: 5, State: 14, Action: 2, Reward: 1.0, Done: True
Prueba conseguida
